## GIỚI THIỆU

#### 1. Nguồn:

- Dataset: [Australian Shark-Incident Database (ASID)](https://taronga.org.au/conservation-and-science/australian-shark-incident-database), do Taronga Conservation Society Australia quản lý. Hợp tác với Flinders University và NSW Department of Primary Industries. 

- Dữ liệu gốc được thu thập từ nhiều nguồn: báo cáo từ nạn nhân hoặc nhân chứng, báo chí, báo cáo chính thức từ các đơn vị thủy sản/bộ/y tế, hồ sơ điều tra, các báo cáo y tế/cảnh sát,... 

#### 2. Giấy phép:

- Dữ liệu công khai được phát hành qua repository chính thức kèm giấy phép mã nguồn mở tại  
  [GitHub – AustralianSharkIncidentDatabase](https://github.com/cjabradshaw/AustralianSharkIncidentDatabase).


- Khi dùng dữ liệu để phân tích/viết báo cáo — cần ghi rõ nguồn “Australian Shark-Incident Database, Taronga Conservation Society Australia” theo yêu cầu của trang web.

#### 3. Về dữ liệu:

- SID tổng hợp các vụ việc “tương tác người – cá mập” ở Úc từ năm 1791 đến nay. Dữ liệu bao gồm khoảng hơn 1000 cá thể vụ việc. 

- Mỗi dòng tương ứng một vụ việc — gồm thông tin về thời gian (tháng, năm), vị trí (latitude, longitude, bang/tiểu bang, địa điểm cụ thể), tình trạng nạn nhân (thương tích, tử vong, không bị thương), loài cá mập nếu biết, hoạt động của nạn nhân lúc xảy ra vụ việc, và nhiều biến khác liên quan: điều kiện nước, thời tiết, hoạt động lặn/câu cá, sự kiện kích động (provoked / unprovoked), thông tin về thiết bị bảo hộ, v.v. 

- Cơ chế ghi nhận dữ liệu: ban đầu từ báo cáo, truyền thông, hồ sơ, sau đó được Taronga xác minh thêm thông qua bảng câu hỏi chi tiết với nạn nhân/nhân chứng hoặc điều tra từ cơ quan chức năng, chuyên gia thủy sản, nhà nghiên cứu. 

- Các vụ việc chỉ được đưa vào database nếu đáp ứng 3 tiêu chí: 
    - (i) người còn sống tại thời điểm xảy ra vụ.
    - (ii) người đang ở trong nước hoặc sử dụng phương tiện nhỏ như kayak, surfboard...
    - (iii) có biểu hiện rõ ràng của cá mập cắn người, thiết bị hoặc phương tiện nhỏ. Có thể ghi lại dù vết cắn không thành công (bumped, attempted bite). 

## DATA CLEANING

In [ ]:
# 01. Import thư viện, load dữ liệu và chuẩn hóa tên cột

import os
import pandas as pd
import numpy as np

# Hàm chuẩn hóa tên cột: bỏ ký tự đặc biệt, khoảng trắng, đổi sang lowercase
def clean_col_name(c):
    c = c.strip()
    c = c.replace(" ", "_").replace(".", "_").replace("/", "_")
    c = c.replace("(", "").replace(")", "").replace("<", "lt_").replace(">", "gt_")
    while "__" in c:
        c = c.replace("__", "_")
    return c.lower().strip("_")

data_path = "data/raw/SharkIncident.csv"
df = pd.read_csv(data_path, low_memory=False)

# Số dòng và cột
print("- Số dòng:", df.shape[0])
print("- Số cột:", df.shape[1])

# Chuẩn hóa tên các cột
df.columns = [clean_col_name(c) for c in df.columns]

print("- 5 dòng đầu:")
display(df.head())

- Số dòng: 1283
- Số cột: 60
- 5 dòng đầu:


,uin,incident_month,incident_year,victim_injury,state,location,latitude,longitude,site_category,site_category_comment,...,spring_or_neap_tide,tidal_cycle,wind_condition,weather_condition,air_temperature_°c,personal_protective_device,deterrent_brand_and_type,data_source,reference,unnamed:_59
0,1,1,1791,fatal,NSW,sydney (near),-33.86,151.2000,coastal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,book,"shark&survl, whitley 1958, book ref 1793",NaN
1,2,3,1803,injured,WA,"shark bay, faure island",-25.8826,113.9226,coastal,bay to open ocean,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,book,"balgridge,green,taylor,whitley 1940",NaN
2,3,1,1807,injured,NSW,"sydney harbour, cockle bay",-33.8661,151.2010,estuary/harbour,bay,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,media outlet,sydney gazette 18.1.1807,NaN
3,4,1,1820,fatal,TAS,"sweetwater point, pitt water",-42.8025,147.4868,estuary/harbour,bay to open ocean,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,witness account,"shark&survl, c. black researcher",NaN
4,5,1,1825,injured,NSW,"sydney harbour, kirribili point",-33.8527,151.2188,estuary/harbour,harbour,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,media outlet,maitland daily mercury 13.11.1899,NaN


In [39]:
# 02. Kiểu dữ liệu của từng cột và số missing values

col_info = pd.DataFrame({
    "dtype": df.dtypes,
    "missing_count": df.isna().sum(),
    "missing_%": round(df.isna().mean() * 100, 2)
})
display(col_info)

,dtype,missing_count,missing_%
uin,int64,0,0.00
incident_month,int64,0,0.00
incident_year,int64,0,0.00
victim_injury,object,0,0.00
state,object,0,0.00
location,object,0,0.00
latitude,object,0,0.00
longitude,float64,0,0.00
site_category,object,0,0.00
site_category_comment,object,558,43.49


Kiểu dữ liệu của các biến `latitude, shark_length_m, victim_age, time_of_incident, time_in_water_min, depth_of_incident_m, total_water_depth_m` hiện đang là object, vì vậy cần chuyển đổi về dạng số trước khi đưa vào phân tích.

In [ ]:
# Các cột object cần chuyển sang dạng số
num_obj_cols = [
    "latitude", "shark_length_m", "victim_age",
    "time_of_incident", "time_in_water_min",
    "depth_of_incident_m", "total_water_depth_m"
]

for col in num_obj_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(",", ".", regex=False)      # đổi dấu phẩy thành dấu chấm
        .str.extract(r"([-+]?[0-9]*\.?[0-9]+)")  # tách giá trị số 
        .astype(float)                                          
    )


| Tên biến                                                                 | Ý nghĩa                                                                         |
| ------------------------------------------------------------------------ | ------------------------------------------------------------------------------- |
| **uin**                                                                  | Mã định danh duy nhất cho mỗi vụ việc.                 |
| **incident_month**                                                       | Tháng xảy ra vụ việc.                              |
| **incident_year**                                                        | Năm xảy ra vụ việc(yyyy).                                                               |
| **victim_injury**                                                        | Loại thương tích (fatal / injured / uninjured)                                 |
| **state**                                                                | Bang/khu vực địa lý nơi vụ việc xảy ra.                                         |
| **location**                                                             | Địa điểm cụ thể hơn của sự cố (bãi biển, vùng biển…).                           |
| **latitude**, **longitude**                                              | Toạ độ địa lý của vụ tấn công.                                                  |
| **site_category**, **site_category_comment**                             | Loại địa điểm (bơi, lặn, câu cá…) + ghi chú bổ sung.                            |
| **shark_common_name**                                                    | Tên thông thường của loài cá mập liên quan tới sự cố.                             |
| **shark_scientific_name**                                                | Tên khoa học của loài cá mập liên quan tới sự cố.                                              |
| **shark_identification_method**                                          | Phương pháp xác định loài (DNA, răng, vết cắn, hành vi…).                          |
| **shark_identification_source**                                          | Nguồn xác nhận thông tin loài (báo cáo, nhân chứng…).                           |
| **shark_length_m**                                                       | Chiều dài ước tính của cá mập (m).                                              |
| **basis_for_length**                                                     | Cơ sở để ước lượng chiều dài (quan sát, đo đạc, camera…).                       |
| **provoked_unprovoked**                                                  | Vụ tấn công mang tính tự nhiên hay bị kích động (unprovoked / provoked).        |
| **provocative_act**                                                      | Hành động cụ thể gây kích động (nếu có).                                        |
| **no_sharks**                                                            | Số lượng cá mập xuất hiện.                                  |
| **victim_activity**                                                      | Nạn nhân đang làm gì tại thời điểm bị tấn công (bơi, lướt ván…).                |
| **fish_speared?**                                                        | Nạn nhân có đang mang theo cá nhỏ hoặc mồi câu không (dễ thu hút cá mập). |
| **commercial_dive_activity**                                             | Hoạt động lặn có mang tính thương mại/tour hay cá nhân.                         |
| **object_of_bite**                                                       | Bộ phận bị cá mập cắn: người, ván lướt, thuyền…                                 |
| **present_at_time_of_bite**                                              | Những gì xuất hiện cạnh nạn nhân (đàn cá, chim biển…).                          |
| **direction_first_strike**                                               | Hướng tiếp cận ban đầu của cá mập.                                              |
| **shark_behaviour**                                                      | Hành vi quan sát được (rình rập, bơi vòng…).                                    |
| **victim_aware_of_shark**                                                | Nạn nhân có phát hiện cá mập trước khi bị cắn hay không.                        |
| **shark_captured**                                                       | Cá mập có bị bắt lại sau sự cố hay không.                                       |
| **injury_location**                                                      | Vị trí thương tích trên cơ thể.                                                 |
| **injury_severity**                                                      | Mức độ thương tích (nhẹ, trung bình, nghiêm trọng, tử vong…).                   |
| **victim_gender**, **victim_age**                                        | Giới tính và độ tuổi nạn nhân.                                                   |
| **victim_clothing**, **clothing_coverage**, **dominant_clothing_colour** | Thông tin trang phục.          |
| **other_clothing_colour**, **clothing_pattern**                          | Màu sắc/phối hoa văn phụ của trang phục.                                   |
| **fin_colour**                                                           | Màu vây của nạn nhân(2 màu = multicolour).                                                   |
| **diversionary_action_taken**                                            | Nạn nhân/xung quanh có hành động xua đuổi không.                                |
| **diversionary_action_outcome**                                          | Kết quả hành động xua đuổi đó.                                                           |
| **people_lt_3m**, **people_3-15m**                                       | Số người có mặt gần nạn nhân trong bán kính 3m hoặc 3–15m.                      |
| **time_of_incident**                                                     | Thời điểm xảy ra sự cố (định dạng 1830).                               |
| **depth_of_incident_m**, **total_water_depth_m**                         | Độ sâu nơi xảy ra sự cố và độ sâu tổng khu vực.                                  |
| **teeth_recovered**                                                      | Có thu hồi được răng/mảnh răng nào không.                 |
| **time_in_water_min**                                                    | Thời gian nạn nhân đã ở dưới nước trước vụ việc (phút).                                |
| **water_temperature_°c**, **air_temperature_°c**                         | Nhiệt độ nước và nhiệt độ không khí.                                            |
| **water_visability_m**                                                   | Tầm nhìn dưới nước (m).                                                         |
| **distance_to_shore_m**                                                  | Khoảng cách từ vị trí bị tấn công tới bờ.                                       |
| **spring_or_neap_tide**, **tidal_cycle**                                 | Trạng thái thuỷ triều.                        |
| **wind_condition**, **weather_condition**                                | Điều kiện gió và thời tiết.                                 |
| **personal_protective_device**                                           | Nạn nhân có mặc đồ bảo hộ hay không.                                         |
| **deterrent_brand_and_type**                                             | Loại thiết bị chống cá mập được sử dụng.                                        |
| **data_source**, **reference**                                           | Nguồn gốc dữ liệu, tài liệu tham khảo.                                          |
| **unnamed:_59**                                                          | Cột rác.                                            |


In [41]:
# 03. Bỏ các cột thiếu dữ liệu và xóa các dòng lặp lại

# Drop cột thiếu > 99% dữ liệu vì các cột này không thể cung cấp insight hữu ích, chỉ tăng nhiễu và làm sai lệch thống kê.
missing_threshold = 0.99
missing_percent = df.isna().sum() / len(df) #missing percent
cols_drop = missing_percent[missing_percent > missing_threshold].index.tolist()
df = df.drop(columns=cols_drop)
print(f"- Xóa {len(cols_drop)} cột:")
print(cols_drop)

# Xử lý duplicated rows
dups = df.duplicated()
print(f"\n- Tổng số dòng trùng lặp: {dups.sum()}")
if dups.any():
    # Loại bỏ các bản ghi trùng lặp chính xác, mỗi dòng phải là một sự kiện duy nhất.
    df = df.drop_duplicates(keep='first')
    print(f"- Đã xóa cách dòng trùng lặp, shape sau khi xóa: {df.shape}")


# Missing values sau khi drop cột
missing_overview = df.isna().sum().sort_values(ascending=False).to_frame("missing_count")
missing_overview["missing_percent"] = (missing_overview["missing_count"] / len(df) * 100).round(2)
print("- Missing value sau xử lí:") 
display(missing_overview.head(15)) #In top 15


- Xóa 5 cột:
['fish_speared?', 'clothing_pattern', 'spring_or_neap_tide', 'deterrent_brand_and_type', 'unnamed:_59']

- Tổng số dòng trùng lặp: 0
- Missing value sau xử lí:


,missing_count,missing_percent
other_clothing_colour,1261,98.29
fin_colour,1251,97.51
tidal_cycle,1249,97.35
air_temperature_°c,1240,96.65
personal_protective_device,1227,95.64
weather_condition,1226,95.56
teeth_recovered,1223,95.32
diversionary_action_outcome,1217,94.86
water_visability_m,1211,94.39
wind_condition,1202,93.69


### Vì sao chọn loại bỏ các cột có >99% missing thay vì xóa dòng?

- Các cột thiếu quá 99% dữ liệu không còn đủ thông tin để phân tích hay huấn luyện mô hình. Giữ lại chỉ làm tăng nhiễu và độ phức tạp.

- Nếu cố gắng giữ lại các cột này, ta gần như phải điền toàn bộ giá trị, khiến biến trở nên kém ý nghĩa và dễ gây sai lệch.

- Việc drop cột giúp giữ lại tối đa số dòng, nếu drop dòng thay vì drop cột thì sẽ làm mất dữ liệu quan sát.


In [42]:
# 04. Xử lý missing dữ liệu

# A. Các cột số
num_cols = df.select_dtypes(include=['number']).columns.tolist()

# Chỉ xử lý âm/0 cho các trường đo đếm không thể âm/0
# Loại latitude, longitude ra vì chúng có thể âm
invalid_numeric_cols = [
    c for c in num_cols
    if not any(x in c.lower() for x in ["latitude", "longitude"])
       and any(k in c.lower() for k in ["length", "age", "distance", "total_water_depth", "visability", "people"])
]

for c in invalid_numeric_cols:
    # giá trị âm chuyển thành missing
    df.loc[df[c] < 0, c] = np.nan
    
    # giá trị 0 (riêng_depth_of_incident_m giữ nguyên được xem là ngay mặt nước) 
    if c.lower() not in ["depth_of_incident_m"]:
        df.loc[df[c] == 0, c] = np.nan

# B. Điền giá trị thiếu bằng -1 cho cột số
df[num_cols] = df[num_cols].fillna(-1)

# C. Điền missing object thành Unknown
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
df[cat_cols] = df[cat_cols].fillna("unknown")


### Vì sao chọn cách thay giá trị này?

- Chọn thay missing value và các giá trị sai (âm hoặc bằng 0) của các cột số thành –1 và thay missing value của các cột phân loại thành “Unknown” vì đây là dữ liệu ghi nhận thực tế. Nếu điền median hoặc mode sẽ tạo ra giá trị không có cơ sở và làm sai lệch dữ liệu gốc.

- Riêng cột depth_of_incident_m vẫn giữ giá trị 0 vì theo ngữ cảnh dữ liệu, 0 được xem là sự cố xảy ra ngay trên mặt nước hoặc sát bờ biển.

In [43]:
# 04. Phân tích outlier

def detect_outliers(series):
    s = series[series != -1]  # bỏ -1 ra khỏi dữ liệu
    Q1 = s.quantile(0.25)
    Q3 = s.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    out = s[(s < lower) | (s > upper)]
    return (len(out), len(out)/len(s)*100, s.min(), s.max(), lower, upper)

result = []
for col in df.select_dtypes(include=np.number).columns:
    count, rate, minv, maxv, low, up = detect_outliers(df[col])
    result.append([col, count, rate, minv, maxv, low, up])

outlier_df = pd.DataFrame(result, columns=[
    " Cột ", " Số lượng outlier ", " Tỷ lệ % ", 
    " Min ", " Max ", " Ngưỡng dưới ", " Ngưỡng trên "
])

print("\Thống kê outlier:")
display(outlier_df)


\Thống kê outlier:


,Cột,Số lượng outlier,Tỷ lệ %,Min,Max,Ngưỡng dưới,Ngưỡng trên
0,uin,0,0.000000,1.0000,1285.0000,-640.00000,1924.00000
1,incident_month,0,0.000000,1.0000,12.0000,-10.00000,22.00000
2,incident_year,3,0.233827,1791.0000,2025.0000,1815.50000,2131.50000
3,latitude,67,5.222136,-43.6523,-9.3809,-47.47435,-11.20235
4,longitude,205,15.978176,96.8192,4034.0000,117.99360,172.70120
5,shark_length_m,0,0.000000,0.3000,6.0000,-0.75000,6.05000
6,no_sharks,21,1.764706,1.0000,10.0000,1.00000,1.00000
7,victim_age,8,1.048493,4.0000,84.0000,-13.00000,67.00000
8,people_lt_3m,6,6.451613,1.0000,12.0000,-0.50000,3.50000
9,people_3-15m,7,8.333333,1.0000,40.0000,-2.00000,6.00000


### Nhận xét:

- uin, incident_month, shark_length_m, time_of_incident, air_temperature_°c không có hoặc có rất ít outlier dữ liệu ổn, hợp lý.

- incident_year có 3 outlier (≈0.23%), có thể là lỗi nhập liệu hoặc các trường hợp ghi chép mốc thời gian không chính xác, mức độ không lớn, không ảnh hưởng nhiều đến phân tích chung.

- longitude có nhiều outlier lớn (≈16%), giá trị vượt xa phạm vi tọa độ địa lý thực tế (-180 đến 180), khả năng cao do lỗi nhập hoặc mã hóa sai, cần xem xét làm sạch.

- victim_age có 8 outlier (≈1%), bao gồm một số trường hợp rất trẻ (4 tuổi) hoặc già (84 tuổi), phản ánh các nạn nhân hiếm gặp.

- depth_of_incident_m có 83 outlier (≈15.7%), phần lớn là các vụ tấn công xảy ra tại vùng nước rất nông hoặc sâu bất thường.

- time_in_water_min có 24 outlier (≈10%), là các nạn nhân ở lâu trong nước hoặc ngắn bất thường.

- no_sharks, people_lt_3m, people_3-15m, water_temperature_°c, water_visability_m, total_water_depth_m các giá trị bị xem là outlier phần lớn là do xuất hiện trong những trường hợp hiếm, không mang tính bất thường.

- distance_to_shore_m có outlier lớn (tối đa ~280 km), phản ánh các vụ tấn công hiếm gặp ở vùng nước cực kì xa bờ.

- latitude có số oulier tương đương distance_to_shore_m, điều này giải thích cho các vụ tấn công hiếm gặp ở những vùng nước cực kì xa bờ.

### Quyết định xử lý:

- Không loại outlier trong hầu hết các cột vì nhiều trường hợp phản ánh tình huống thực tế của sự kiện tấn công cá mập.

- Chỉ xem xét longitude vì giá trị vượt ngưỡng địa lý không hợp lệ.

- Các cột như distance_to_shore_m, latitude, depth_of_incident_m, time_in_water_min, total_water_depth_m giữ nguyên giá trị outlier vì phản ánh các tình huống thực tế hiếm gặp, cần thiết cho phân tích hành vi và bối cảnh tấn công.


In [44]:
df.loc[df["longitude"] > 180, "longitude"] = -1
df.loc[df["longitude"] < -180, "longitude"] = -1

In [45]:
# 06. Lưu dữ liệu

out_dir = "data/processed"
os.makedirs(out_dir, exist_ok=True)
out_path = os.path.join(out_dir, "SharkIncident_cleaned.csv")
df.to_csv(out_path, index=False)
print(f"\nĐã lưu tập dữ liệu tại: {out_path}")



Đã lưu tập dữ liệu tại: data/processed\SharkIncident_cleaned.csv
